In [ ]:
from tabulate import tabulate
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='LMS_p',
)

cursor = connection.cursor()

def issue_book():
    print("Enter book issuance details:")
    try:
        member_id = int(input("Enter the member ID: "))
        book_id = int(input("Enter the book ID: "))
        
        # Check if the member exists
        sel_member = "SELECT * FROM Member WHERE Member_id = %s"
        cursor.execute(sel_member, (member_id,))
        member_record = cursor.fetchone()
        
        # Check if the book exists and has available copies
        sel_book = "SELECT * FROM Book1 WHERE Book_id = %s AND Copies > 0"
        cursor.execute(sel_book, (book_id,))
        book_record = cursor.fetchone()
        
        if member_record and book_record:
            issue_id = int(input("Enter an issue ID: "))
            issue_date = input("Enter a date in YYYY-MM-DD format: ")
            copies1 = int(input("Enter the number of copies to issue: "))
            
            if copies1 > 0 and copies1 <= book_record[4]:  # Assuming Copies is at index 4
                # Insert into issue table
                ins_issue = 'INSERT INTO Issue1(Issue_id, Issue_date, Member_id1, Book_id1, Copies1) VALUES (%s, %s, %s, %s, %s)'
                ins_values_issue = (issue_id, issue_date, member_id, book_id, copies1)
                cursor.execute(ins_issue, ins_values_issue)
                
                # Update Rem_copies in the book table
                update_copies = "UPDATE Book1 SET Rem_copies = Copies - %s WHERE Book_id = %s"
                cursor.execute(update_copies, (copies1, book_id))
                
                connection.commit()
                print("Book issuance successful!")
            else:
                print("Invalid number of copies or insufficient available copies.")
        else:
            print('No such member exists or no available copies of the book')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")

def return_book():
    print("Enter book return details:")
    try:
        # Fetch all issued books
        cursor.execute("SELECT * FROM Issue1")
        issued_books = cursor.fetchall()

        # Display all issued books
        print("\nIssued Books:")
        print(tabulate(issued_books, headers=['Issue_id', 'Issue_date', 'Member_id1', 'Book_id1', 'Copies1']))

        issue_id = int(input("Enter the issue ID from the list above to return the book: "))

        # Check if the issued book exists
        sel_issue = "SELECT * FROM Issue1 WHERE Issue_id = %s"
        cursor.execute(sel_issue, (issue_id,))
        issued_record = cursor.fetchone()

        if issued_record:
            return_id = int(input("Enter a return ID: "))
            return_date = input("Enter a date in YYYY-MM-DD format: ")
            copies_returned = int(input("Enter the number of copies to return: "))
            
            if copies_returned > 0 and copies_returned <= issued_record[4]:  # Assuming Copies1 is at index 4
                # Insert into Return1 table
                ins_return = 'INSERT INTO Return1(Return_id, Return_date, Member_id2, Book_id2, Copies2) VALUES (%s, %s, %s, %s, %s)'
                ins_values_return = (return_id, return_date, issued_record[2], issued_record[3], copies_returned)  # Assuming Member_id1 is at index 2, and Book_id1 is at index 3
                cursor.execute(ins_return, ins_values_return)

                # Update Rem_copies in the Book table
                update_copies = "UPDATE Book1 SET Rem_copies = Rem_copies + %s WHERE Book_id = %s"
                cursor.execute(update_copies, (copies_returned, issued_record[3]))  # Assuming Book_id1 is at index 3

                # Delete the entry from the Issue1 table
                del_issue = "DELETE FROM Issue1 WHERE Issue_id = %s"
                cursor.execute(del_issue, (issue_id,))

                connection.commit()
                print("Book return successful!")
            else:
                print("Invalid number of copies or insufficient copies issued for the book.")
        else:
            print('No such issued book exists with the provided issue ID')

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")
#return_book()

In [1]:
from tabulate import tabulate
import mysql.connector

connection = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root',
    database='LMS_p',
)

cursor = connection.cursor()

def available_books():
    try:
        # Fetch only available books with copies
        cursor.execute("SELECT * FROM Book1 WHERE rem_copies > 0")
        books_with_copies = cursor.fetchall()

        if not books_with_copies:
            print("No available books with copies.")
            return

        # Display available books with all details
        print("\nAvailable Books:")
        print(tabulate(books_with_copies, headers=['Book_id', 'Bname', 'Author', 'Price', 'Copies', 'Rem_copies']))

    except mysql.connector.Error as error:
        print(f"MySQL error: {error}")

